In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_experimental.text_splitter import SemanticChunker
from langchain.document_loaders import UnstructuredExcelLoader
from pymongo import MongoClient
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
import os

# 1. 엑셀 파일 로드 및 문서로 변환
xlsx_path = 'test용사전엑셀.xlsx'
xlsx_loader = UnstructuredExcelLoader(xlsx_path, mode="elements")
docs = xlsx_loader.load()  # 로드된 원본 문서

# 2. Embeddings 및 문서 분할 설정
embeddings = OpenAIEmbeddings()
semantic_chunker = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")
semantic_chunks = semantic_chunker.create_documents([d.page_content for d in docs])

# 3. MongoDB 연결 설정
MONGODB_ATLAS_CLUSTER_URI = os.getenv("MONGODB_ATLAS_CLUSTER_URI")
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)

db_name = "test"
collection_name = "col"

# MongoDB 컬렉션 객체 가져오기
collection = client[db_name][collection_name]

# 4. Vector Search 초기화 (chunked 문서 사용)
vector_search = MongoDBAtlasVectorSearch.from_documents(
    documents=semantic_chunks,  # **chunker로 분할된 문서 사용**
    embedding=embeddings,      # Embedding 모델
    collection=collection      # MongoDB 컬렉션
)


In [8]:
# 번역을 위한 PromptTemplate 설정
translation_prompt = PromptTemplate(
    input_variables=["retrieved_text", "query"],
    template=""" 
다음은 한국어 텍스트와 관련된 참고 자료입니다:
참고 자료: {retrieved_text}

위 자료를 기반으로, 아래 한국어 문장을 영어로 번역해주세요.
문장: {query}
번역:
"""
)

# LLM 모델 설정 (OpenAI GPT-4)
trans_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 번역 체인 설정
translation_chain = LLMChain(
    llm=trans_llm, 
    prompt=translation_prompt
)

# MongoDB에서 관련 데이터를 검색
query = "산독끼는 바나나와 꼬라니를 좋아해."

# MongoDB에서 검색하여 관련 텍스트 추출
words = query.split()  # 쿼리에서 단어들로 분할

# DB에서 해당 단어에 대한 번역 데이터를 찾는 부분
retrieved_text = ""
for word in words:
    result = collection.find_one({"text": {"$regex": word}})
    if result:
        english_translation = result.get('text', '').split()  # 해당 단어의 번역 결과
        retrieved_text += " ".join(english_translation) + " "

# LLM으로 번역 결과 얻기
result = translation_chain.run({"retrieved_text": retrieved_text.strip(), "query": query})

# 번역 결과 출력
print("번역 결과:", result)

번역 결과: The translation of the sentence "산독끼는 바나나와 꼬라니를 좋아해." is "Sandoqki likes bananas and corani."
